In [12]:
import pandas as pd
from catboost import CatBoost, Pool
from sklearn.metrics import ndcg_score
from sklearn.model_selection import train_test_split

In [8]:
# Загрузка данных
data = pd.read_csv('/content/drive/MyDrive/Colab Content/intern_task.csv')


In [13]:
# Подсчет количества элементов в каждом классе query_id
query_id_counts = data['query_id'].value_counts()

# Отфильтровать данные, исключив классы с одним элементом
filtered_data = data[data['query_id'].isin(query_id_counts[query_id_counts > 1].index)]

# Сортировка данных по query_id
filtered_data.sort_values(by='query_id', inplace=True)

# Подготовка данных
X = filtered_data.drop(['rank'], axis=1)  # Удаляем только столбец 'rank'
y = filtered_data['rank']
query_id = filtered_data['query_id']

# Получение уникальных query_id
unique_query_ids = query_id.unique()

# Разделение уникальных query_id на обучающую и тестовую выборки
train_query_ids, test_query_ids = train_test_split(unique_query_ids, test_size=0.2, random_state=42)

# Разделение данных на обучающую и тестовую выборки на основе query_id
X_train = X[X['query_id'].isin(train_query_ids)]
y_train = y[X['query_id'].isin(train_query_ids)]
X_test = X[X['query_id'].isin(test_query_ids)]
y_test = y[X['query_id'].isin(test_query_ids)]

# Обучение модели CatBoost
train_pool = Pool(data=X_train, label=y_train, group_id=X_train['query_id'])
test_pool = Pool(data=X_test, label=y_test, group_id=X_test['query_id'])


<ipython-input-13-127b66102cc4>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data.sort_values(by='query_id', inplace=True)


In [14]:
model = CatBoost({'loss_function': 'YetiRank', 'iterations': 100, 'custom_metric': ['NDCG'], 'verbose': False})
model.fit(train_pool)

In [15]:
# Предсказание рангов на тестовой выборке
y_pred = model.predict(test_pool)

# Расчет метрик ранжирования (NDCG@5)
ndcg_5 = ndcg_score(y_test.values.reshape(1, -1), y_pred.reshape(1, -1), k=5)

print(f"NDCG@5: {ndcg_5}")

NDCG@5: 0.9152099486815958
